In [1]:
from ast import literal_eval

import numpy as np
import pandas as pd
import scipy.sparse

from lightfm import LightFM
from lightfm.data import Dataset
from recommender.testing.custom_metric_utils import create_recommendations_profiles_embeddings, calculate_metric_scores
from recommender.testing.custom_metric_utils import create_users_profiles_embeddings
from recommender.testing.dataset_utils import prepare_interactions
from recommender.tools.lightfm_utils import LightFMTests

In [2]:
DATA_ROOT = '../../data'
THREADS = 8

## Loading data

In [3]:
test_df = pd.read_csv(f'{DATA_ROOT}/ratings_test_implicit.csv.gz')
train_df = pd.read_csv(f'{DATA_ROOT}/ratings_train_implicit.csv.gz')
games_df = pd.read_json(f'{DATA_ROOT}/bgg_GameItem.jl', lines=True)[[
    'name', 'bgg_id', 'mechanic', 'category', 'complexity',
    'max_players_best', 'min_players_best', 'max_players_rec', 'min_players_rec'
]]

features_names = pd.read_csv(f'{DATA_ROOT}/game_features_names.csv.gz').values.flatten()
game_features = pd.read_csv(f'{DATA_ROOT}/game_features.csv.gz')

In [44]:
users_profiles = pd.read_pickle(f'{DATA_ROOT}/users_profiles.pkl')
games_profiles = pd.read_pickle(f'{DATA_ROOT}/games_profiles.pkl')

In [5]:
mechanics_names = features_names[:20]
categories_names = features_names[20:40]

## Adding new users

In [6]:
USERS_TO_ADD = {
    # Family: Catan, Monopoly, UNO, Ticket to Ride
    "USER_1": [13, 1406, 2223, 9209],
    # Negative interactions: Game of Thrones, War of the Ring, Twilight Imperium (3rd edition),
    # Battlestar Galactica, Scythe
    "USER_2": [103343, 2228, 12493, 37111, 169786],
    # Worker placement: Viticulture, Agricola, Feast for Odin, West Kingdom
    "USER_3": [183394, 31260, 177736, 236457, 266810, 296151],
    # Our games: Spirit Island, Citadel, Splendor, Avalon, Codenames, Crew, Love Letter, 7 wonders, viticulture, Descent
    "USER_4": [162886, 478, 148228, 1862, 178900, 324856, 129622, 68448, 183394, 150586],
    # Ameritrash: Runewars, Arkham Horror, Starcraft, Last Night on Earth, Talisman, Divine Right
    "USER_5": [59294, 15987, 22827, 29368, 27627, 23],
    # Eurogames: Catan, Puerto Rico, Carcassone, Ra, El Grande, Five Tribes
    "USER_6": [13, 3076, 822, 12, 93, 157354],
}

In [7]:
def append_users_to_train_df(train_df, users):
    for user, games in users.items():
        train_df = train_df.append(pd.DataFrame(data={'bgg_user_name': [user]*len(games), 'bgg_id': games, 'value':[1]*len(games)}))
    return train_df

In [45]:
train_df = append_users_to_train_df(train_df, USERS_TO_ADD)
profiles_to_add = create_users_profiles_embeddings(
    train_df, games_df, categories_names, mechanics_names, 10, show_progress=True, users_subset=[f'USER_{i}' for i in range(1,7)]
)
users_profiles = users_profiles.append(profiles_to_add)

100%|██████████| 6/6 [00:00<00:00, 232.50it/s]


## Preparing interactions, item features & profiles

In [9]:
full_df = pd.concat([train_df, test_df], ignore_index=True)
dataset = Dataset()
print("Fitting dataset")
dataset.fit((x for x in full_df['bgg_user_name']), (x for x in full_df['bgg_id']),
            item_features=(x for x in features_names))

Fitting dataset


In [10]:
game_features['features'] = game_features.features.apply(literal_eval)
games_list = full_df['bgg_id'].unique()
game_features = game_features.drop((game_features['bgg_id'])[~game_features['bgg_id'].isin(games_list)].index)
item_features = dataset.build_item_features((val['bgg_id'], val['features'] + [val['bgg_id']])
                                        for idx, val in game_features.iterrows())

In [11]:
try:
    train_interactions = scipy.sparse.load_npz('../notebooks_data/custom_train_interactions.npz')
    test_interactions = scipy.sparse.load_npz('../notebooks_data/custom_test_interactions.npz')
except:
    train_interactions, test_interactions = prepare_interactions(train_df, test_df, dataset)
    scipy.sparse.save_npz('../notebooks_data/custom_train_interactions.npz', train_interactions)
    scipy.sparse.save_npz('../notebooks_data/custom_test_interactions.npz', test_interactions)

Preparing training interactions
Preparing testing interactions


In [46]:
users_profiles.index = users_profiles.index.map(dataset.mapping()[0])
games_profiles.index = games_profiles.index.map(dataset.mapping()[2])
users_profiles = users_profiles.sort_index()
games_profiles = games_profiles.sort_index()

In [47]:
users_profiles.values[:, :40] *= 3

In [14]:
cython_users_profiles = np.ascontiguousarray(users_profiles.values, dtype=np.float32)
cython_games_profiles = np.ascontiguousarray(games_profiles.values, dtype=np.float32)

## Model training

In [15]:
PARAMS = {'no_components': 70,
          'learning_schedule': 'adadelta',
          'loss': 'custom',
          'item_alpha': 3e-06,
          'max_sampled': 10,
          'rho': 0.93,
          'epsilon': 1.0e-06,
          'random_state': 42}

model = LightFM(**PARAMS)
model.fit_partial(
    train_interactions, verbose=True, item_features=item_features, epochs=104,
    num_threads=THREADS, user_profiles=cython_users_profiles, item_profiles=cython_games_profiles
)

Epoch: 100%|██████████| 104/104 [1:27:05<00:00, 50.24s/it]


## Inspecting predictions

In [16]:
tests = LightFMTests(model=model,
                     train_df=train_df,
                     dataset=dataset,
                     train_interactions=train_interactions,
                     test_interactions=test_interactions,
                     item_features=item_features)

In [55]:
def analyze(user_name):
    def game_id_to_name(game_id, games_df):
        game_info = games_df.loc[games_df['bgg_id'] == game_id]
        return game_info
    top_5 = tests.get_top_n_for_user_by_name(user_name=user_name, training_games_ids=USERS_TO_ADD[user_name], n=5)
    top_20 = tests.get_top_n_for_user_by_name(user_name=user_name, training_games_ids=USERS_TO_ADD[user_name], n=20)
    top_5_df = pd.DataFrame({"bgg_user_name": user_name, "bgg_id": top_5}).explode('bgg_id').reset_index(drop=True)
    recommendations_profile = create_recommendations_profiles_embeddings(top_5_df, 5, games_df, categories_names, mechanics_names, 10)
    print("Metric value:", calculate_metric_scores(recommendations_profile, users_profiles.iloc[dataset.mapping()[0][user_name]:dataset.mapping()[0][user_name]+1]))
    top_20_names = [game_id_to_name(game_id, games_df) for game_id in top_20]
    return pd.concat(top_20_names)

In [56]:
# Family: Catan, Monopoly, UNO, Ticket to Ride
analyze("USER_1")

Metric value: [2.25       1.41428571 1.         0.95639445 5.62068017]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
1095,Monopoly: Star Wars,1298,"[Auction/Bidding:2012, Player Elimination:2685...","[Economic:1021, Movies / TV / Radio theme:1064...",1.7609,8.0,2.0,8.0,2.0
23210,Monopoly: Star Wars Limited Collector's Edition,26696,"[Auction/Bidding:2012, Player Elimination:2685...","[Economic:1021, Movies / TV / Radio theme:1064...",1.6548,8.0,2.0,8.0,2.0
2928,Monopoly: The Simpsons,3394,"[Auction/Bidding:2012, Player Elimination:2685...","[Economic:1021, Movies / TV / Radio theme:1064...",1.6154,6.0,2.0,6.0,2.0
7044,Monopoly: The Lord of the Rings Trilogy Edition,8041,"[Auction/Bidding:2012, Player Elimination:2685...","[Economic:1021, Movies / TV / Radio theme:1064...",1.5833,6.0,2.0,6.0,2.0
4351,Monopoly: Star Wars Episode I,5029,"[Auction/Bidding:2012, Player Elimination:2685...","[Economic:1021, Movies / TV / Radio theme:1064...",1.8276,8.0,2.0,8.0,2.0
33089,Byzanz,38032,"[Auction/Bidding:2012, Auction: Turn Order Unt...","[Ancient:1050, Arabian:1052, Card Game:1002, E...",1.6145,6.0,3.0,6.0,3.0
5884,Attack!,6752,"[Action Points:2001, Area Movement:2046, Dice ...","[Economic:1021, Miniatures:1047, Negotiation:1...",2.5472,6.0,2.0,6.0,2.0
6204,Monopoly: Deluxe Edition,7098,"[Auction/Bidding:2012, Player Elimination:2685...","[Economic:1021, Negotiation:1026]",1.8904,10.0,2.0,10.0,2.0
597,Monopoly: The Card Game,684,"[Set Collection:2004, Trading:2008]","[Card Game:1002, Economic:1021, Negotiation:1026]",1.4681,6.0,2.0,6.0,2.0
1548,Trump: The Game,1824,"[Auction/Bidding:2012, Memory:2047, Roll / Spi...","[Economic:1021, Memory:1045, Movies / TV / Rad...",1.7000,4.0,3.0,4.0,3.0


In [57]:
# Negative interactions: Game of Thrones, War of the Ring, Twilight Imperium (3rd edition),
# Battlestar Galactica, Scythe
analyze("USER_2")

Metric value: [1.95       1.62857143 0.68       0.93797174 5.19654317]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
42845,Eclipse,72125,"[Area Majority / Influence:2080, Dice Rolling:...","[Civilization:1015, Fighting:1046, Science Fic...",3.7002,6.0,4.0,6.0,2.0
63412,Star Wars: Imperial Assault,164153,"[Dice Rolling:2072, Die Icon Resolution:2856, ...","[Adventure:1022, Exploration:1020, Fighting:10...",3.3099,5.0,2.0,5.0,2.0
27625,BattleTech Introductory Box Set,31759,"[Dice Rolling:2072, Hexagon Grid:2026, Variabl...","[Fighting:1046, Miniatures:1047, Science Ficti...",3.5395,6.0,2.0,6.0,2.0
80878,Eclipse: Second Dawn for the Galaxy,246900,"[Alliances:2916, Area Majority / Influence:208...","[Civilization:1015, Science Fiction:1016, Spac...",3.5341,6.0,4.0,6.0,2.0
69475,Star Wars: Rebellion,187645,"[Area Majority / Influence:2080, Area Movement...","[Civil War:1102, Fighting:1046, Miniatures:104...",3.7252,2.0,2.0,2.0,2.0
2401,Car Wars,2795,"[Dice Rolling:2072, Grid Movement:2676, Impuls...","[Fighting:1046, Miniatures:1047, Racing:1031, ...",3.0976,4.0,4.0,8.0,2.0
73466,The Edge: Dawnfall,207729,"[Area Majority / Influence:2080, Cooperative G...","[Fantasy:1010, Fighting:1046, Miniatures:1047,...",3.0000,2.0,2.0,4.0,1.0
53975,Duel of Ages II,129459,"[Grid Movement:2676, Hexagon Grid:2026, Line o...","[Adventure:1022, Fantasy:1010, Fighting:1046, ...",3.4111,4.0,2.0,6.0,2.0
25340,Dust,29109,"[Area Majority / Influence:2080, Dice Rolling:...","[Miniatures:1047, Science Fiction:1016, Wargam...",2.6133,6.0,4.0,6.0,2.0
1443,Gladiator,1693,"[Action Queue:2689, Critical Hits and Failures...","[Ancient:1050, Economic:1021, Fighting:1046, W...",2.9184,6.0,2.0,6.0,2.0


In [58]:
# Worker placement: Viticulture, Agricola, Feast for Odin, West Kingdom
analyze("USER_3")

Metric value: [2.4        1.95       0.75       0.70490904 5.80490904]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
55718,New Dawn,135654,"[Area Majority / Influence:2080, Dice Rolling:...","[Economic:1021, Science Fiction:1016, Space Ex...",3.0244,4.0,2.0,4.0,2.0
72165,Agricola (Revised Edition),200680,"[Enclosure:2043, Hand Management:2040, Worker ...","[Animals:1089, Economic:1021, Farming:1013]",3.5147,4.0,4.0,4.0,1.0
75260,Empires of the Void II,218509,"[Action Points:2001, Area Majority / Influence...","[Civilization:1015, Economic:1021, Exploration...",3.4556,4.0,3.0,5.0,2.0
64506,Letter Tycoon,169147,"[Commodity Speculation:2013, Hand Management:2...","[Economic:1021, Word Game:1025]",2.0400,5.0,2.0,5.0,2.0
58316,Three Kingdoms Redux,145371,"[Area Majority / Influence:2080, Auction/Biddi...","[Ancient:1050, Civilization:1015, Economic:102...",4.0385,3.0,3.0,3.0,3.0
22259,If Wishes Were Fishes!,25584,"[Area Majority / Influence:2080, Card Drafting...","[Animals:1089, Economic:1021]",1.9487,3.0,3.0,5.0,2.0
87376,Anachrony: Infinity Box,278292,"[Action Points:2001, Card Drafting:2041, Dice ...","[Economic:1021, Miniatures:1047, Science Ficti...",4.0000,4.0,1.0,4.0,1.0
64197,Terraforming Mars,167791,"[Drafting:2984, End Game Bonuses:2875, Hand Ma...","[Economic:1021, Environmental:1084, Industry /...",3.2428,3.0,3.0,4.0,1.0
88513,The Magnificent,283863,"[Card Drafting:2041, Contracts:2912, Dice Roll...","[Dice:1017, Economic:1021]",3.1792,4.0,1.0,4.0,1.0
65408,Mombasa,172386,"[Action Queue:2689, Area Majority / Influence:...",[Economic:1021],3.8961,4.0,4.0,4.0,2.0


In [59]:
# Our games: Spirit Island, Citadel, Splendor, Avalon, Codenames, Crew, Love Letter, 7 wonders, viticulture, Descent
analyze("USER_4")

Metric value: [1.92857143 1.38461538 0.75294118 0.84243732 4.90856531]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
95909,7 Wonders (Second Edition),316377,"[Card Drafting:2041, Drafting:2984, Hand Manag...","[Ancient:1050, Card Game:1002, City Building:1...",2.1778,5.0,4.0,7.0,3.0
378,Alhambra: The Card Game,431,"[Area Majority / Influence:2080, Card Drafting...","[Arabian:1052, Card Game:1002, City Building:1...",1.7234,6.0,2.0,6.0,2.0
74435,Food Truck Champion,213661,"[Card Drafting:2041, Follow:2843, Hand Managem...","[Card Game:1002, Economic:1021, Industry / Man...",2.5556,5.0,2.0,5.0,2.0
51571,Cover Your Assets,121193,"[Hand Management:2040, Set Collection:2004, Ta...","[Card Game:1002, Economic:1021]",1.1379,6.0,2.0,6.0,2.0
33089,Byzanz,38032,"[Auction/Bidding:2012, Auction: Turn Order Unt...","[Ancient:1050, Arabian:1052, Card Game:1002, E...",1.6145,6.0,3.0,6.0,3.0
86133,It's a Wonderful World,271324,"[Card Drafting:2041, Drafting:2984, End Game B...","[Card Game:1002, Civilization:1015, Economic:1...",2.2895,4.0,4.0,5.0,1.0
37979,Legacy: The Testament of Duke de Crecy,52461,"[Card Drafting:2041, End Game Bonuses:2875, Ha...","[Age of Reason:2726, Card Game:1002, Economic:...",2.7080,4.0,1.0,4.0,1.0
52161,City of Iron,123499,"[Action Points:2001, Area Majority / Influence...","[Card Game:1002, City Building:1029, Civilizat...",3.2353,4.0,4.0,4.0,2.0
78051,Century: Golem Edition,232832,"[Action Retrieval:2839, Card Drafting:2041, Co...","[Card Game:1002, Economic:1021, Fantasy:1010]",1.6706,3.0,3.0,5.0,2.0
78105,Carthago: Merchants & Guilds,233006,"[Action Points:2001, Deck, Bag, and Pool Build...","[Ancient:1050, Card Game:1002, Economic:1021]",2.9565,4.0,2.0,4.0,2.0


In [60]:
# Ameritrash: Runewars, Arkham Horror, Starcraft, Last Night on Earth, Talisman, Divine Right
analyze("USER_5")

Metric value: [2.44       1.32       0.8        0.70534209 5.26534209]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
3083,Mordheim: City of the Damned,3565,"[Dice Rolling:2072, Measurement Movement:2949,...","[Exploration:1020, Fantasy:1010, Fighting:1046...",3.0882,4.0,2.0,6.0,2.0
61167,Krosmaster: Quest,155827,"[Action Points:2001, Dice Rolling:2072, Grid M...","[Adventure:1022, Exploration:1020, Fantasy:101...",3.0909,6.0,2.0,6.0,2.0
62359,Dungeon Saga: Dwarf King's Quest,160081,"[Dice Rolling:2072, Grid Movement:2676, Modula...","[Adventure:1022, Exploration:1020, Fantasy:101...",2.4773,5.0,2.0,5.0,2.0
71779,"Heroes of Land, Air & Sea",198830,"[Action Points:2001, Area Majority / Influence...","[Exploration:1020, Fantasy:1010, Fighting:1046...",3.3140,4.0,4.0,5.0,2.0
63412,Star Wars: Imperial Assault,164153,"[Dice Rolling:2072, Die Icon Resolution:2856, ...","[Adventure:1022, Exploration:1020, Fighting:10...",3.3099,5.0,2.0,5.0,2.0
60983,Arcadia Quest,155068,"[Dice Rolling:2072, Grid Movement:2676, Modula...","[Adventure:1022, Exploration:1020, Fantasy:101...",2.5352,4.0,4.0,4.0,2.0
36822,Chaos in the Old World,43111,"[Action Points:2001, Area Majority / Influence...","[Fantasy:1010, Fighting:1046, Horror:1024, Myt...",3.1892,4.0,4.0,4.0,3.0
2401,Car Wars,2795,"[Dice Rolling:2072, Grid Movement:2676, Impuls...","[Fighting:1046, Miniatures:1047, Racing:1031, ...",3.0976,4.0,4.0,8.0,2.0
67540,Arcadia Quest: Inferno,179803,"[Dice Rolling:2072, Grid Movement:2676, Modula...","[Adventure:1022, Fantasy:1010, Fighting:1046, ...",2.4091,4.0,4.0,4.0,2.0
6549,WarCraft: The Board Game,7479,"[Dice Rolling:2072, Modular Board:2011, Team-B...","[City Building:1029, Fantasy:1010, Fighting:10...",2.7567,4.0,4.0,4.0,2.0


In [61]:
# Eurogames: Catan, Puerto Rico, Carcassone, Ra, El Grande, Five Tribes
analyze("USER_6")

Metric value: [2.4        0.91578947 0.68571429 0.7365691  4.73807286]


,name,bgg_id,mechanic,category,complexity,max_players_best,min_players_best,max_players_rec,min_players_rec
66458,504,175878,"[Area Majority / Influence:2080, Area Movement...","[Animals:1089, City Building:1029, Economic:10...",3.4700,4.0,4.0,4.0,2.0
47811,The Gnomes of Zavandor,103184,"[Card Drafting:2041, Commodity Speculation:201...","[Economic:1021, Fantasy:1010]",2.5789,4.0,2.0,4.0,2.0
47735,Caverna: The Cave Farmers,102794,"[Automatic Resource Growth:2903, Increase Valu...","[Animals:1089, Economic:1021, Fantasy:1010, Fa...",3.7898,4.0,4.0,5.0,1.0
1106,Die Magier von Pangea,1309,"[Modular Board:2011, Pick-up and Deliver:2007,...","[Economic:1021, Fantasy:1010]",2.3276,4.0,2.0,4.0,2.0
78452,Mystery of the Temples,234691,"[Action Queue:2689, Area Majority / Influence:...","[Economic:1021, Fantasy:1010]",2.4000,4.0,2.0,4.0,2.0
73066,Key to the City: London,205507,"[Auction/Bidding:2012, Modular Board:2011, Set...","[City Building:1029, Economic:1021, Territory ...",2.7222,6.0,2.0,6.0,2.0
12177,The Scepter of Zavandor,13884,"[Auction/Bidding:2012, Set Collection:2004, Va...","[Economic:1021, Fantasy:1010]",3.5100,4.0,4.0,5.0,2.0
48237,Archipelago,105551,"[Area Majority / Influence:2080, Auction/Biddi...","[Civilization:1015, Economic:1021, Exploration...",3.7323,4.0,4.0,5.0,2.0
72215,Pixie Queen,200890,"[Auction/Bidding:2012, Worker Placement:2082]","[Economic:1021, Fantasy:1010, Mythology:1082]",3.1905,4.0,3.0,5.0,3.0
75201,Rise to Nobility,218293,"[Dice Rolling:2072, Set Collection:2004, Varia...","[City Building:1029, Dice:1017, Economic:1021,...",3.2917,6.0,1.0,4.0,1.0
